In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa

In [3]:
ds = xr.open_dataset('/bsuhome/zacharykeskinen/scratch/data/uavsar/ncs/model/model.re.nc')

pa.PyAPS

In [5]:
lvls = np.array([1, 2, 3, 5, 7, 10, 20, 30, 50, 70, 100, 125, 150, 175, 
                     200, 225, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 775, 
                     800, 825, 850, 875, 900, 925, 950, 975, 1000])
nlvls = len(lvls)
gphind = np.arange(nlvls)*12+1

In [6]:
lvls = 100.0*lvls                 #Conversion to absolute pressure
for i in range(nlvls):
    print(gphind[i])

1
13
25
37
49
61
73
85
97
109
121
133
145
157
169
181
193
205
217
229
241
253
265
277
289
301
313
325
337
349
361
373
385
397
409
421
433
